## 1.Word Embedding

In [ ]:
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

# 关于word embedding, 以序列建模为例
# 考虑source sentence 和 target sentence
# 构建序列, 序列的字符以其在词表中的索引的形式表示
batch_size = 2

# 单词表大小
max_num_src_words = 8
max_num_tgt_words = 8

model_dim = 4  # embedding维度

# 序列的最大长度
max_src_seq_len = 5
max_tgt_seq_len = 5

# src_len = torch.randint(2, 5, (batch_size,))
# tgt_len = torch.randint(2, 5, (batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)

# 单词索引构成源句子和目标句子,构建batch， 并且做了padding, 默认值为0
# src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max_src_seq_len-L)), 0) for L in src_len])

sequences = []
for L in src_len:
    seq = torch.randint(1, max_num_src_words, (L,)) # 随机序列
    seq_padded = F.pad(seq, (0, max_src_seq_len - L)) # 右填充
    sequences.append(torch.unsqueeze(seq_padded, 0)) # 增加batch维度
src_seq = torch.cat(sequences, 0) # 拼接成batch


# tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max_tgt_seq_len-L)), 0) for L in tgt_len])

sequences = []
for L in tgt_len:
    seq = torch.randint(1, max_num_tgt_words, (L,)) # 随机序列
    seq_padded = F.pad(seq, (0, max_tgt_seq_len - L)) # 右填充
    sequences.append(torch.unsqueeze(seq_padded, 0)) # 增加batch维度
tgt_seq = torch.cat(sequences, 0) # 拼接成batch

# 构造embedding
src_Embedding_table = nn.Embedding(max_num_src_words + 1, model_dim)  # +1是因为0号索引是padding
tgt_Embedding_table = nn.Embedding(max_num_tgt_words + 1, model_dim)  # +1是因为0号索引是padding
src_Embedding = src_Embedding_table(src_seq)
tgt_Embedding = tgt_Embedding_table(tgt_seq)

print(src_Embedding_table.weight)
print(src_seq)
print(src_Embedding)


Parameter containing:
tensor([[-0.9632,  0.6525,  1.2353,  1.6336],
        [-0.8140,  0.3155, -0.3363, -0.8945],
        [ 0.4698, -0.8110,  0.4505, -0.2385],
        [-0.8686,  1.0179, -0.7343,  1.1894],
        [-1.3481, -2.0940,  0.3460,  1.0921],
        [ 1.6486, -0.6314,  1.3420,  1.5739],
        [ 1.3399, -0.8183,  0.5063,  1.2263],
        [-0.2652, -1.8749, -0.5610, -1.6820],
        [ 0.2110,  1.3561, -0.4643, -0.4526]], requires_grad=True)
tensor([[4, 3, 0, 0, 0],
        [7, 4, 1, 1, 0]])
tensor([[[-1.3481, -2.0940,  0.3460,  1.0921],
         [-0.8686,  1.0179, -0.7343,  1.1894],
         [-0.9632,  0.6525,  1.2353,  1.6336],
         [-0.9632,  0.6525,  1.2353,  1.6336],
         [-0.9632,  0.6525,  1.2353,  1.6336]],

        [[-0.2652, -1.8749, -0.5610, -1.6820],
         [-1.3481, -2.0940,  0.3460,  1.0921],
         [-0.8140,  0.3155, -0.3363, -0.8945],
         [-0.8140,  0.3155, -0.3363, -0.8945],
         [-0.9632,  0.6525,  1.2353,  1.6336]]], grad_fn=<Embedding

### 总结

Word Embedding 演示了序列建模中如何构建词嵌入：

```
词索引序列 → Padding 对齐 → Embedding 查表 → 稠密向量
```

**步骤：**

| 步骤 | 代码 | 说明 |
|------|------|------|
| 1. 定义参数 | `batch_size=2, model_dim=4` | 2 个样本，嵌入维度 4 |
| 2. 生成序列 | `torch.randint(1, 8, (L,))` | 随机生成词索引 |
| 3. Padding | `F.pad(seq, (0, max_len-L))` | 右侧填 0，对齐长度 |
| 4. 组 batch | `torch.cat(..., dim=0)` | 拼成 `(batch, seq_len)` |
| 5. 嵌入 | `nn.Embedding(9, 4)` | 查表得到向量 |

**形状变化：**

```
src_seq:       (2, 5)      # 2 个句子，每句 5 个词索引
src_Embedding: (2, 5, 4)   # 每个词变成 4 维向量
```

## 2.Position Embedding

In [9]:
import math

# 正弦位置编码
# pos: 位置索引 (0, 1, 2, ...)
# i: 维度索引
# PE(pos, 2i)   = sin(pos / 10000^(2i/d))
# PE(pos, 2i+1) = cos(pos / 10000^(2i/d))

def sinusoidal_position_embedding(max_len, dim):
    """
    生成正弦位置编码
    max_len: 最大序列长度
    dim: embedding维度
    返回: (max_len, dim) 的位置编码矩阵
    """
    pe = torch.zeros(max_len, dim)
    pos = torch.arange(0, max_len).unsqueeze(1).float()  # (max_len, 1)
    # 计算分母: 10000^(2i/d) = exp(2i * log(10000) / d)
    div = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pe[:, 0::2] = torch.sin(pos * div)  # 偶数维度用sin
    pe[:, 1::2] = torch.cos(pos * div)  # 奇数维度用cos
    return pe

# 生成位置编码
src_pos_embedding = sinusoidal_position_embedding(max_src_seq_len, model_dim)  # (5, 4)
tgt_pos_embedding = sinusoidal_position_embedding(max_tgt_seq_len, model_dim)  # (5, 4)

# Word Embedding + Position Embedding
src_input = src_Embedding + src_pos_embedding  # (2, 5, 4) + (5, 4) 广播
tgt_input = tgt_Embedding + tgt_pos_embedding  # (2, 5, 4) + (5, 4) 广播

print("位置编码 shape:", src_pos_embedding.shape)
print("位置编码:\n", src_pos_embedding)
print("\n最终输入 shape:", src_input.shape)
print("最终输入 (Word + Position):\n", src_input)

位置编码 shape: torch.Size([5, 4])
位置编码:
 tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.0400,  0.9992]])

最终输入 shape: torch.Size([2, 5, 4])
最终输入 (Word + Position):
 tensor([[[-1.3481e+00, -1.0940e+00,  3.4596e-01,  2.0921e+00],
         [-2.7103e-02,  1.5582e+00, -7.2429e-01,  2.1893e+00],
         [-5.3930e-02,  2.3636e-01,  1.2553e+00,  2.6334e+00],
         [-8.2211e-01, -3.3748e-01,  1.2653e+00,  2.6331e+00],
         [-1.7200e+00, -1.1328e-03,  1.2753e+00,  2.6328e+00]],

        [[-2.6524e-01, -8.7486e-01, -5.6096e-01, -6.8205e-01],
         [-5.0660e-01, -1.5537e+00,  3.5596e-01,  2.0921e+00],
         [ 9.5303e-02, -1.0065e-01, -3.1635e-01,  1.0531e-01],
         [-6.7287e-01, -6.7449e-01, -3.0635e-01,  1.0506e-01],
         [-1.7200e+00, -1.1328e-03,  1.2753e+00,  2.6328e+00]]],
       grad_fn=<AddBackward0>)


### 总结

Position Embedding 为序列注入位置信息，与 Word Embedding 相加作为 Transformer 输入：

```
Word Embedding + Position Embedding = 最终输入
   (语义信息)       (位置信息)
```

**正弦位置编码公式：**
- 偶数维度：$PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d})$
- 奇数维度：$PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d})$

**代码步骤：**

| 步骤 | 代码 | 说明 |
|------|------|------|
| 1. 位置索引 | `pos = torch.arange(0, max_len).unsqueeze(1)` | 形状 (seq_len, 1) |
| 2. 频率分母 | `div = torch.exp(torch.arange(0, dim, 2) * -(log(10000)/dim))` | 等价于 1/10000^(2i/d) |
| 3. 填充 sin/cos | `pe[:, 0::2] = sin`, `pe[:, 1::2] = cos` | 偶数列 sin，奇数列 cos |
| 4. 广播相加 | `src_Embedding + src_pos_embedding` | (2,5,4) + (5,4) → (2,5,4) |

**为什么用 sin/cos 成对？**
- 可通过线性变换表示相对位置：$PE_{pos+k} = M \cdot PE_{pos}$
- 模型能学习"词与词之间的距离"

**形状变化：**
```
src_pos_embedding: (5, 4)      # 位置编码
src_Embedding:     (2, 5, 4)   # 词嵌入
src_input:         (2, 5, 4)   # 广播相加，最终输入
```

## 3.Encoder Self-Attention Mask

In [ ]:
# Encoder Self-Attention Mask
# 目的：屏蔽 padding 位置，不让它们参与注意力计算

# 第一步：构建有效位置向量 (batch, seq_len)
# 1 = 有效位置，0 = padding
valid_encoder_pos = torch.cat([
    torch.unsqueeze(
        torch.cat([torch.ones(L), torch.zeros(max_src_seq_len - L)]), 0
    ) 
    for L in src_len
])

print("有效位置向量 shape:", valid_encoder_pos.shape)
print("有效位置向量:\n", valid_encoder_pos)
# 样本0: [1, 1, 0, 0, 0]  前2个有效
# 样本1: [1, 1, 1, 1, 0]  前4个有效

# 第二步：扩展成矩阵 (batch, seq_len, seq_len)
# 外积: (batch, seq_len, 1) × (batch, 1, seq_len)
valid_encoder_pos_matrix = torch.bmm(
    valid_encoder_pos.unsqueeze(-1),  # (2, 5, 1)，2个（5，1）的列向量
    valid_encoder_pos.unsqueeze(1)    # (2, 1, 5)，2个（1，5）的行向量
)

print("\nEncoder Self-Attention Mask shape:", valid_encoder_pos_matrix.shape)
print("样本0的mask:\n", valid_encoder_pos_matrix[0])
print("样本1的mask:\n", valid_encoder_pos_matrix[1])

# 转换为 bool 类型，后续用于屏蔽
encoder_self_attn_mask = valid_encoder_pos_matrix.bool()
print("\n最终mask (bool):\n", encoder_self_attn_mask)

有效位置向量 shape: torch.Size([2, 5])
有效位置向量:
 tensor([[1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0.]])

Encoder Self-Attention Mask shape: torch.Size([2, 5, 5])
样本0的mask:
 tensor([[1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
样本1的mask:
 tensor([[1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]])

最终mask (bool):
 tensor([[[ True,  True, False, False, False],
         [ True,  True, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[ True,  True,  True,  True, False],
         [ True,  True,  True,  True, False],
         [ True,  True,  True,  True, False],
         [ True,  True,  True,  True, False],
         [False, False, False, False, False]]])


### 总结

Encoder Self-Attention Mask 用于屏蔽 padding 位置，防止无意义的填充参与注意力计算：

```
有效位置 → 正常计算注意力
padding  → 屏蔽（softmax 后权重为 0）
```

**代码步骤：**

| 步骤 | 代码 | 说明 |
|------|------|------|
| 1. 有效位置向量 | `torch.cat([ones(L), zeros(max_len-L)])` | 1=有效，0=padding |
| 2. 扩展成矩阵 | `torch.bmm(pos.unsqueeze(-1), pos.unsqueeze(1))` | 外积得到 (batch, seq, seq) |
| 3. 转布尔类型 | `.bool()` | 方便后续 masked_fill |

**外积原理：**
```
列向量 (5,1) × 行向量 (1,5) → 矩阵 (5,5)

[1]                     [[1,1,0,0,0],
[1]   × [1,1,0,0,0]  =   [1,1,0,0,0],
[0]                      [0,0,0,0,0],
[0]                      [0,0,0,0,0],
[0]                      [0,0,0,0,0]]
```

**Mask 含义：**
- `mask[i][j] = True`：query_i 可以看 key_j
- `mask[i][j] = False`：屏蔽，query_i 不能看 key_j

**形状变化：**
```
valid_encoder_pos:        (2, 5)      # 有效位置向量
valid_encoder_pos_matrix: (2, 5, 5)   # 注意力 mask 矩阵
```

## 4.Intra-Attention Mask

In [11]:
# Intra-Attention Mask (Cross-Attention)
# 目的：Decoder 看 Encoder 时，屏蔽双方的 padding

# Decoder 有效位置 (batch, tgt_seq_len)
valid_decoder_pos = torch.cat([
    torch.unsqueeze(
        torch.cat([torch.ones(L), torch.zeros(max_tgt_seq_len - L)]), 0
    ) 
    for L in tgt_len
])

print("Encoder 有效位置:", valid_encoder_pos)  # src_len = [2, 4]
print("Decoder 有效位置:", valid_decoder_pos)  # tgt_len = [4, 3]

# Cross-Attention Mask: (batch, tgt_seq_len, src_seq_len)
# Decoder query (tgt) × Encoder key (src)
cross_attn_mask = torch.bmm(
    valid_decoder_pos.unsqueeze(-1),  # (2, 5, 1)
    valid_encoder_pos.unsqueeze(1)    # (2, 1, 5)
).bool()

print("\nCross-Attention Mask shape:", cross_attn_mask.shape)
print("样本0的mask (tgt_len=4, src_len=2):\n", cross_attn_mask[0].int())
print("样本1的mask (tgt_len=3, src_len=4):\n", cross_attn_mask[1].int())

Encoder 有效位置: tensor([[1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0.]])
Decoder 有效位置: tensor([[1., 1., 1., 1., 0.],
        [1., 1., 1., 0., 0.]])

Cross-Attention Mask shape: torch.Size([2, 5, 5])
样本0的mask (tgt_len=4, src_len=2):
 tensor([[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)
样本1的mask (tgt_len=3, src_len=4):
 tensor([[1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)


### 总结

Cross-Attention 让 Decoder 查询 Encoder 的信息，Mask 屏蔽双方的 padding：

```
Decoder (Q) → 发起查询："源句子哪部分和我相关？"
Encoder (KV) → 被查询：提供源句子的语义信息
```

**与 Self-Attention 的区别：**

| 类型 | Query | Key/Value | Mask 形状 |
|------|-------|-----------|-----------|
| Self-Attention | 自己 | 自己 | (batch, seq, seq) |
| Cross-Attention | Decoder | Encoder | (batch, tgt_len, src_len) |

**代码步骤：**

| 步骤 | 代码 | 说明 |
|------|------|------|
| 1. Decoder 有效位置 | `valid_decoder_pos` | tgt_len = [4, 3] |
| 2. Encoder 有效位置 | `valid_encoder_pos` | src_len = [2, 4] |
| 3. 外积得 Mask | `bmm(decoder.unsqueeze(-1), encoder.unsqueeze(1))` | (batch, tgt_len, src_len) |

**Mask 含义：**
- `mask[i][j] = True`：Decoder 位置 i 可以看 Encoder 位置 j
- `mask[i][j] = False`：屏蔽（任一方是 padding）

**形状变化：**
```
valid_decoder_pos: (2, 5)       # Decoder 有效位置
valid_encoder_pos: (2, 5)       # Encoder 有效位置
cross_attn_mask:   (2, 5, 5)    # Cross-Attention Mask
```

## 5.Decoder Self-Attention Mask

In [ ]:
# Decoder Self-Attention Mask
# 目的：1. 屏蔽 padding  2. 屏蔽未来位置（不能偷看答案）

# 第一步：Causal Mask（因果掩码）- 下三角矩阵
# 每个位置只能看自己和之前的位置
causal_mask = torch.tril(torch.ones(max_tgt_seq_len, max_tgt_seq_len))
print("Causal Mask (下三角):")
print(causal_mask)

# 第二步：Padding Mask - 有效位置矩阵
valid_decoder_pos_matrix = torch.bmm(
    valid_decoder_pos.unsqueeze(-1),  # (2, 5, 1)
    valid_decoder_pos.unsqueeze(1)    # (2, 1, 5)
)
print("\nPadding Mask 样本0 (tgt_len=4):")
print(valid_decoder_pos_matrix[0])

# 第三步：两者相乘（同时满足两个条件）
decoder_self_attn_mask = (causal_mask * valid_decoder_pos_matrix).bool() # 哈达玛积，对应元素相乘

print("\nDecoder Self-Attention Mask shape:", decoder_self_attn_mask.shape)
print("样本0的mask (tgt_len=4):\n", decoder_self_attn_mask[0].int())
print("样本1的mask (tgt_len=3):\n", decoder_self_attn_mask[1].int())

Causal Mask (下三角):
tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

Padding Mask 样本0 (tgt_len=4):
tensor([[1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]])

Decoder Self-Attention Mask shape: torch.Size([2, 5, 5])
样本0的mask (tgt_len=4):
 tensor([[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)
样本1的mask (tgt_len=3):
 tensor([[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=torch.int32)


### 总结

Decoder Self-Attention Mask 需要同时满足两个约束：

```
1. Causal Mask：不能看未来（防止作弊）
2. Padding Mask：不能看 padding（无意义填充）
```

**代码步骤：**

| 步骤 | 代码 | 说明 |
|------|------|------|
| 1. Causal Mask | `torch.tril(torch.ones(seq_len, seq_len))` | 下三角矩阵 |
| 2. Padding Mask | `bmm(pos.unsqueeze(-1), pos.unsqueeze(1))` | 有效位置矩阵 |
| 3. 相乘 | `causal_mask * padding_mask` | Hadamard Product，同时满足两个条件 |

**三种 Mask 对比：**

| Mask | 约束 | 形状 |
|------|------|------|
| Encoder Self-Attention | 只屏蔽 padding | (batch, src_len, src_len) |
| Cross-Attention | 屏蔽双方 padding | (batch, tgt_len, src_len) |
| Decoder Self-Attention | padding + 未来 | (batch, tgt_len, tgt_len) |

**Causal Mask 示意：**
```
位置0: [1, 0, 0, 0, 0]  ← 只能看自己
位置1: [1, 1, 0, 0, 0]  ← 能看 0, 1
位置2: [1, 1, 1, 0, 0]  ← 能看 0, 1, 2
...
```

**形状变化：**
```
causal_mask:           (5, 5)      # 下三角
valid_decoder_pos_matrix: (2, 5, 5)   # padding mask
decoder_self_attn_mask:   (2, 5, 5)   # 最终 mask
```

## 6.Multi-Head Self-Attention

In [19]:
# Multi-Head Self-Attention
# 核心公式: Attention(Q,K,V) = softmax(QK^T / sqrt(d_k)) * V

import math

# 参数设置
num_heads = 2
head_dim = model_dim // num_heads  # 4 // 2 = 2

print(f"model_dim={model_dim}, num_heads={num_heads}, head_dim={head_dim}")

# 四个线性层: W_Q, W_K, W_V, W_O
W_Q = nn.Linear(model_dim, model_dim)
W_K = nn.Linear(model_dim, model_dim)
W_V = nn.Linear(model_dim, model_dim)
W_O = nn.Linear(model_dim, model_dim)

# 第一步: 计算 Q, K, V
Q = W_Q(src_input)  # (batch, seq_len, model_dim) = (2, 5, 4)
K = W_K(src_input)
V = W_V(src_input)
print("Q shape:", Q.shape)

# 第二步: 拆成多头
# (batch, seq_len, model_dim) → (batch, seq_len, num_heads, head_dim) → (batch, num_heads, seq_len, head_dim)
Q = Q.view(batch_size, -1, num_heads, head_dim).transpose(1, 2)  # -1自动计算seq_len
K = K.view(batch_size, -1, num_heads, head_dim).transpose(1, 2)
V = V.view(batch_size, -1, num_heads, head_dim).transpose(1, 2)
print("Q after reshape:", Q.shape)  # (2, 2, 5, 2)

# 第三步: 计算注意力分数 QK^T / sqrt(d_k)
scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(head_dim)
print("Attention scores shape:", scores.shape)  # (2, 2, 5, 5)
print(scores[0, 0])  # 样本0，头0

# 第四步: 应用 Mask (扩展 mask 维度以匹配多头)
# encoder_self_attn_mask: (batch, seq_len, seq_len) → (batch, 1, seq_len, seq_len)
mask_expanded = encoder_self_attn_mask.unsqueeze(1)  # (2, 1, 5, 5)
scores = scores.masked_fill(~mask_expanded, -1e9)
print("Scores after mask:\n", scores[0, 0])  # 样本0，头0

# 第五步: Softmax 得到注意力权重
attn_weights = F.softmax(scores, dim=-1)
print("Attention weights shape:", attn_weights.shape)
print("Attention weights (样本0, 头0):\n", attn_weights[0, 0])

# 第六步: 加权求和 weights * V
attn_output = torch.matmul(attn_weights, V)
print("Attention output shape:", attn_output.shape)  # (2, 2, 5, 2)

# 第七步: 拼接多头
# (batch, num_heads, seq_len, head_dim) → (batch, seq_len, num_heads, head_dim) → (batch, seq_len, model_dim)
attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, model_dim)
print("After concat shape:", attn_output.shape)  # (2, 5, 4)

# 第八步: 输出投影
output = W_O(attn_output)
print("Final output shape:", output.shape)  # (2, 5, 4)

model_dim=4, num_heads=2, head_dim=2
Q shape: torch.Size([2, 5, 4])
Q after reshape: torch.Size([2, 2, 5, 2])
Attention scores shape: torch.Size([2, 2, 5, 5])
tensor([[-0.2136,  0.1260, -0.1885, -0.2376, -0.1703],
        [-0.1974, -0.2793, -0.3767, -0.3894, -0.5325],
        [-0.7051, -0.0283, -0.8493, -0.9746, -0.9829],
        [-0.6870,  0.1070, -0.7586, -0.8918, -0.8301],
        [-0.8434,  0.2553, -0.8680, -1.0418, -0.9018]],
       grad_fn=<SelectBackward0>)
Scores after mask:
 tensor([[-2.1362e-01,  1.2597e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [-1.9742e-01, -2.7935e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],
       grad_fn=<SelectBackward0>)
Attention weights shape: torch.Size([2, 2, 5, 5])
Attention weights (样本0, 头0):
 tensor([[0.4159, 0.58